In [1]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# 重み付き平均MAPE表の作成@CG

In [2]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

/tmp/ipykernel_165/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_165/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [3]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

# 重み付き平均MAPE表の作成@MG

In [4]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

mg_input_list_problem_size: list[int] = [32, 64, 128, 256]
mg_input_list_nit: list[int] = [4, 10, 20, 35]

mg_target_list_problem_size: list[int] = [512]
mg_target_list_nit: list[int] = [50]

# input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=input_list_process,
#     list_na=cg_input_list_na,
#     list_nonzer=cg_input_list_nonzer,
#     list_niter=cg_input_list_niter,
#     list_shift=cg_input_list_shift,
#     csvDir=csvDirPath,
# )
# input_rawDF_cg = input_rawDF_cg.rename(columns={"Name":"functionName"})

input_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=input_list_process,
    list_problem_size=mg_input_list_problem_size,
    list_nit=mg_input_list_nit,
    csvDir=csvDirPath,
)
input_rawDF_mg = input_rawDF_mg.rename(columns={"Name": "functionName"})


# target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=target_list_process,
#     list_na=cg_target_list_na,
#     list_nonzer=cg_target_list_nonzer,
#     list_niter=cg_target_list_niter,
#     list_shift=cg_target_list_shift,
#     csvDir=csvDirPath,
# )
# target_rawDF_cg = target_rawDF_cg.rename(columns={"Name":"functionName"})

target_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=target_list_process,
    list_problem_size=mg_target_list_problem_size,
    list_nit=mg_target_list_nit,
    csvDir=csvDirPath,
)
target_rawDF_mg = target_rawDF_mg.rename(columns={"Name": "functionName"})

mg_list_exp: list[str] = ["process", "problem_size", "nit"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_mg["functionName"])))

In [5]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_mg[
        input_rawDF_mg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_mg[
        target_rawDF_mg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=mg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")
    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[mg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_mg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [6]:
retNum_mg

38.136058981254756